# Single-Step Function Calling with OCI Generative AI

### What this file does:
Demonstrates single-step function calling using OCI Generative AI Cohere models. Shows how to define tools, make tool calls, and provide tool results to get a final response, with both regular and streaming versions.

**Documentation to reference:**
- OCI Gen AI: https://docs.oracle.com/en-us/iaas/Content/generative-ai/pretrained-models.htm
- Cohere Command Models: https://docs.cohere.com/docs/command-r
- OCI Python SDK: https://github.com/oracle/oci-python-sdk/tree/master/src/oci/generative_ai_inference

**Relevant slack channels:**
- #generative-ai-users: *for questions on OCI Gen AI*
- #igiu-innovation-lab: *general discussions on your project*
- #igiu-ai-learning: *help with sandbox environment or help with running this code*

**Env setup:**
- sandbox.yaml: Contains OCI config, compartment, and other details.
- .env: Load environment variables (e.g., API keys if needed).
- configure cwd for jupyter match your workspace python code: 
    -  vscode menu -> Settings > Extensions > Jupyter > Notebook File Root
    -  change from `${fileDirname}` to `${workspaceFolder}`


**How to run in notebook:**
- Make sure your runtime environment has all dependencies and access to required config files.
- Run the notebook cells in order.

### Supported models (https://docs.oracle.com/en-us/iaas/Content/generative-ai/chat-models.htm)
- cohere.command-a-03-2025
- cohere.command-r-08-2024
- cohere.command-r-plus-08-2024


In [ ]:
# set up the  variables

from oci.generative_ai_inference import GenerativeAiInferenceClient
from oci.generative_ai_inference.models import OnDemandServingMode, EmbedTextDetails,CohereChatRequest, ChatDetails
import oci
import json, os
from dotenv import load_dotenv
from envyaml import EnvYAML

#####
#make sure your sandbox.yaml file is setup for your environment. You might have to specify the full path depending on  your `cwd` 
# you can also try making your cwd ofr jupyter match your workspace python code: 
# vscopde menu -> Settings > Extensions > Jupyter > Notebook File Root
# change from ${fileDirname} to ${workspaceFolder}
#####

SANDBOX_CONFIG_FILE = "sandbox.yaml"
load_dotenv()

LLM_MODEL = "cohere.command-a-03-2025" 
PREAMBLE = """
        Analyze the problem and pick the right set of tools to answer the question
"""
MESSAGE = """
       "I'd like 4 apples and a fish please"
"""
llm_service_endpoint= "https://inference.generativeai.us-chicago-1.oci.oraclecloud.com"

## Sep up the tools

In [ ]:
item_param = oci.generative_ai_inference.models.CohereParameterDefinition()
item_param.description = "the item requested to be purchased, in all caps eg. Bananas should be BANANAS"
item_param.type = "str"
item_param.is_required = True

quantity_param = oci.generative_ai_inference.models.CohereParameterDefinition()
quantity_param.description = "how many of the items should be purchased"
quantity_param.type = "int"
quantity_param.is_required = True

shop_tool = oci.generative_ai_inference.models.CohereTool()
shop_tool.name = "personal_shopper"
shop_tool.description = "Returns items and requested volumes to purchase"
shop_tool.parameter_definitions = {
    "item": item_param,
    "quantity": quantity_param
}

## read the config file 

In [ ]:
scfg = EnvYAML(SANDBOX_CONFIG_FILE)
if scfg is None or "oci" not in scfg or "bucket" not in scfg:
    raise RuntimeError("Invalid sandbox configuration.")

#read the oci config
config = oci.config.from_file(os.path.expanduser(scfg["oci"]["configFile"]),scfg["oci"]["profile"])
            

## specify the tools to use in the chat request 

In [ ]:
# chat request      
llm_chat_request = CohereChatRequest()
llm_chat_request.preamble_override = PREAMBLE 
llm_chat_request.message = MESSAGE
llm_chat_request.is_stream = False 
llm_chat_request.max_tokens = 500 # max token to generate, can lead to incomplete responses
llm_chat_request.is_force_single_step = True
llm_chat_request.tools = [ shop_tool]



# set up chat details
chat_detail = ChatDetails()
chat_detail.serving_mode = OnDemandServingMode(model_id=LLM_MODEL)
chat_detail.compartment_id = scfg["oci"]["compartment"]
chat_detail.chat_request = llm_chat_request



## call the LLM 

In [ ]:
# set up the LLM client 
llm_client = GenerativeAiInferenceClient(
                config=config,
                service_endpoint=llm_service_endpoint,
                retry_strategy=oci.retry.NoneRetryStrategy(),
                timeout=(10,240))

step = 1
chat_response = llm_client.chat(chat_detail)
print(f"**************************Step {step} Result**************************")
print(f"message = {chat_response.data.chat_response.text}")
print(f"tool calls = {chat_response.data.chat_response.tool_calls}")

## Call the tool

Note: 
1. in this example we are not explicity calling the tool, we are just returning a made up response.  you will insert an explicit call to teh toolapi for real code
2. We have the call teh llm again with teh tool response for the final answer

In [ ]:
llm_chat_request.tool_results = []
i = 0
for call in chat_response.data.chat_response.tool_calls:
        tool_result = oci.generative_ai_inference.models.CohereToolResult()
        tool_result.call = call
        # try to change response to out of stock etc for one or both items and see
        tool_result.outputs = [ { "response": "Completed, in stock" } ] 
        llm_chat_request.tool_results.append(tool_result)

chat_response = llm_client.chat(chat_detail)

# Print result
print("**************************Step 2 Result**************************")
print(f"message = {chat_response.data.chat_response.text}")
print(f"tool calls = {chat_response.data.chat_response.tool_calls}")

# Streaming version 

Steaming response reduces latency, specially if response has a lot of text. but its involved as we have to process events 

we first define the function to process the events


PS: this seems to be broken. refer to python code ( command_r_tool_single_step_demo_streaming.py) for the implementation

In [ ]:
def get_tool_calls(chat_response):
    for event in chat_response.data.events():
        res = json.loads(event.data)
        text = res['text']
        if 'finishReason' in res:
            if 'toolCalls' in res:
                #print(f"\ntools to use : {res['toolCalls']}",flush=True)
                return text,res['toolCalls']
            else:
                return text,None
        else:
            if 'text' in res:
                print(res['text'], end="", flush=True)
    print("\n")
    return None,None



### call the llm in streaming mode 


In [ ]:
llm_chat_request.is_stream = True
llm_chat_request.tool_results = None
step =1 
chat_response = llm_client.chat(chat_detail)

text,tool_calls= get_tool_calls(chat_response)
print(f"\n **************************Step {step} Result**************************")
print(f"message = {text}")
print(f"tool calls = {tool_calls}")



# call tools

In [ ]:
llm_chat_request.tool_results = []
for call in tool_calls:
    tool_result = oci.generative_ai_inference.models.CohereToolResult()
    tool_result.call = call
    tool_result.outputs = [ { "response": "Completed" } ] 
    llm_chat_request.tool_results.append(tool_result)

step = step+1
chat_response = llm_client.chat(chat_detail)
text,tool_calls= get_tool_calls(chat_response)
print(f"message = {text}")
print(f"tool calls = {tool_calls}")


## Exercise : Weather Answering App

1. Create an App that  gives the weather information for a given city 
    * Eg:  	A. Lucknow will have temp around 25 degrees and will rain 

2. calls the weather API for a given city
    * Create an API 
        * Input 
            * City Name  
            * DatDays in future
        * Output
            * Low
            * High
            * Chance of rain 
3. Types of questions supported
    * What is the weather in lucknow tomm
    * What will be highs in capital of india
    * Which major city in Karnataka will it rain tomm 
